In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

import seaborn as sns

import openpyxl

import re

In [2]:
#import top10k
data_frame1 = pd.read_csv("/Users/mattia/Downloads/top10k-spotify-artist-metadata.csv")
data_frame1.head()

,Unnamed: 0,index,artist,gender,age,type,country,city_1,district_1,city_2,district_2,city_3,district_3
0,0,0,Drake,male,33,person,CA,NaN,NaN,Toronto,NaN,NaN,NaN
1,1,1,Post Malone,male,25,person,US,NaN,NaN,Syracuse,NaN,NaN,NaN
2,2,2,Ed Sheeran,male,29,person,GB,NaN,NaN,Halifax,NaN,NaN,NaN
3,3,3,J Balvin,male,35,person,CO,NaN,NaN,Medellín,NaN,NaN,NaN
4,4,4,Bad Bunny,male,26,person,PR,NaN,NaN,San Juan,NaN,NaN,NaN


In [3]:
#drop unnecessary columns   
gender = data_frame1.drop(columns=['Unnamed: 0', 'city_1', 'district_1', 'city_2', 'district_2', 'city_3', 'district_3'])
gender

,index,artist,gender,age,type,country
0,0,Drake,male,33,person,CA
1,1,Post Malone,male,25,person,US
2,2,Ed Sheeran,male,29,person,GB
3,3,J Balvin,male,35,person,CO
4,4,Bad Bunny,male,26,person,PR
...,...,...,...,...,...,...
9995,4991,si_el_bien,NaN,0,NaN,NaN
9996,4992,Antonello Venditti,male,71,person,IT
9997,4993,Lea Salonga,female,49,person,PH
9998,4994,Vertical Horizon,mixed,29,group,NaN


In [4]:
#Drop 'type' == 'person'
gender = gender[gender.type == 'person']
gender

,index,artist,gender,age,type,country
0,0,Drake,male,33,person,CA
1,1,Post Malone,male,25,person,US
2,2,Ed Sheeran,male,29,person,GB
3,3,J Balvin,male,35,person,CO
4,4,Bad Bunny,male,26,person,PR
...,...,...,...,...,...,...
9993,4989,Nasty C,male,23,person,ZA
9994,4990,Klara Hammarström,female,20,person,NaN
9996,4992,Antonello Venditti,male,71,person,IT
9997,4993,Lea Salonga,female,49,person,PH


In [5]:
#drop duplicates under 'artist'   9832 rows x 6 columns
gender = gender.drop_duplicates(subset=['artist'])
gender

,index,artist,gender,age,type,country
0,0,Drake,male,33,person,CA
1,1,Post Malone,male,25,person,US
2,2,Ed Sheeran,male,29,person,GB
3,3,J Balvin,male,35,person,CO
4,4,Bad Bunny,male,26,person,PR
...,...,...,...,...,...,...
9993,4989,Nasty C,male,23,person,ZA
9994,4990,Klara Hammarström,female,20,person,NaN
9996,4992,Antonello Venditti,male,71,person,IT
9997,4993,Lea Salonga,female,49,person,PH


In [6]:
#drop rows with at least one 'NaN'   6349 rows x 6 columns
gender = gender.dropna()
gender

,index,artist,gender,age,type,country
0,0,Drake,male,33,person,CA
1,1,Post Malone,male,25,person,US
2,2,Ed Sheeran,male,29,person,GB
3,3,J Balvin,male,35,person,CO
4,4,Bad Bunny,male,26,person,PR
...,...,...,...,...,...,...
9992,4988,Daniela Romo,female,60,person,MX
9993,4989,Nasty C,male,23,person,ZA
9996,4992,Antonello Venditti,male,71,person,IT
9997,4993,Lea Salonga,female,49,person,PH


In [8]:
#Download File with explicit content
data_frame2 = pd.read_excel("/Users/mattia/Erasmus/Premaster/Python/data_for_project.xlsm")
data_frame2

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Gender
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,751,834,1,-5444,0,437,3.0,0.001770,355,894,95053,pop,female
1,blink-182,All The Small Things,167066,False,1999,79,434,897,0,-4918,1,488,103.0,0.000000,612,684,148726,"rock, pop",mixed
2,Faith Hill,Breathe,250546,False,1999,66,529,496,7,-9007,1,29,173.0,0.000000,251,278,136859,"pop, country",NaN
3,Bon Jovi,It's My Life,224493,False,2000,78,551,913,0,-4063,0,466,263.0,0.001350,347,544,119992,"rock, metal",mixed
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,614,928,8,-4806,0,516,408.0,104.000000,845,879,172656,pop,mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Jonas Brothers,Sucker,181026,False,2019,79,842,734,1,-5065,0,588,427.0,0.000000,106,952,137958,pop,mixed
1996,Taylor Swift,Cruel Summer,178426,False,2019,78,552,702,9,-5707,1,157,117.0,0.002060,105,564,169994,pop,female
1997,Blanco Brown,The Git Up,200593,False,2019,69,847,678,9,-8635,1,109,669.0,0.000000,274,811,97984,"hip hop, country",male
1998,Sam Smith,Dancing With A Stranger (with Normani),171029,False,2019,75,741,52,8,-7513,1,656,45.0,0.000197,222,347,102998,pop,other


In [20]:
#Drop unnecessary columns
explicit = data_frame2.drop(columns=['duration_ms', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'energy'])
#drop duplicates
explicit = explicit.drop_duplicates(subset=['artist'])
#drop NaN
explicit = explicit.dropna()
explicit

,artist,song,explicit,year,popularity,danceability,genre,Gender
0,Britney Spears,Oops!...I Did It Again,False,2000,77,751,pop,female
1,blink-182,All The Small Things,False,1999,79,434,"rock, pop",mixed
3,Bon Jovi,It's My Life,False,2000,78,551,"rock, metal",mixed
4,*NSYNC,Bye Bye Bye,False,2000,65,614,pop,mixed
5,Sisqo,Thong Song,True,1999,69,706,"hip hop, pop, R&B",male
...,...,...,...,...,...,...,...,...
1982,Kehlani,Nights Like This (feat. Ty Dolla $ign),True,2019,74,61,"hip hop, pop, R&B",female
1983,Sech,Otro Trago,True,2019,71,746,latin,male
1992,Normani,Motivation,False,2019,71,599,"pop, R&B",female
1993,Joel Corry,Sorry,False,2019,63,744,"pop, Dance/Electronic",male


In [26]:
#save rows for which values in column 'explicit' = TRUE in dataframe 'exT' 
exT = explicit[(explicit["explicit"] == True)]
exT

#reset index
exT = exT.reset_index()
exT

,index,artist,song,explicit,year,popularity,danceability,genre,Gender
0,5,Sisqo,Thong Song,True,1999,69,706,"hip hop, pop, R&B",male
1,6,Eminem,The Real Slim Shady,True,2000,86,949,hip hop,male
2,19,Dr. Dre,The Next Episode,True,1999,82,922,hip hop,male
3,30,DMX,Party Up,True,1999,71,51,"hip hop, pop",male
4,44,Missy Elliott,Hot Boyz,True,1998,49,727,"hip hop, pop, R&B",female
...,...,...,...,...,...,...,...,...,...
136,1966,Blueface,Thotiana,True,2018,1,906,hip hop,male
137,1967,DaBaby,Suge,True,2019,72,876,hip hop,male
138,1979,Megan Thee Stallion,Hot Girl Summer (feat. Nicki Minaj & Ty Dolla ...,True,2019,69,872,"hip hop, pop, R&B",female
139,1982,Kehlani,Nights Like This (feat. Ty Dolla $ign),True,2019,74,61,"hip hop, pop, R&B",female


In [46]:
#save rows for which values in column 'explicit' = False in dataframe 'exF'
exF = explicit[(explicit['explicit'] == False)]
exF = exF.reset_index()
exF

,index,artist,song,explicit,year,popularity,danceability,genre,Gender
0,0,Britney Spears,Oops!...I Did It Again,False,2000,77,751,pop,female
1,1,blink-182,All The Small Things,False,1999,79,434,"rock, pop",mixed
2,3,Bon Jovi,It's My Life,False,2000,78,551,"rock, metal",mixed
3,4,*NSYNC,Bye Bye Bye,False,2000,65,614,pop,mixed
4,7,Robbie Williams,Rock DJ,False,2000,68,708,"pop, rock",male
...,...,...,...,...,...,...,...,...,...
393,1977,Dominic Fike,3 Nights,False,2018,78,815,"rock, pop",male
394,1981,Sam Feldt,Post Malone (feat. RANI),False,2019,69,59,"pop, Dance/Electronic",male
395,1992,Normani,Motivation,False,2019,71,599,"pop, R&B",female
396,1993,Joel Corry,Sorry,False,2019,63,744,"pop, Dance/Electronic",male


In [29]:
#First we will looop throught exT

#create empty list 'frequency', loop through all rows of 'exT', split values of genre, either create new key or add 1 as value
frequency = {}
max = len(exT)
for x in range(max):
    items = re.split(",", exT.loc[int(x):int(x),"genre"].values[0])
    #print (x,items)
    for item in items:
        #print (item)
        if item in frequency:
            frequency[item] += 1
        else:
            frequency[item] = 1

In [30]:
frequency

{'hip hop': 111,
 ' pop': 79,
 ' R&B': 19,
 'set()': 1,
 'pop': 16,
 'rock': 3,
 'R&B': 1,
 ' Dance/Electronic': 12,
 'Folk/Acoustic': 1,
 ' rock': 1,
 'Dance/Electronic': 5,
 'metal': 1,
 'latin': 2}

In [36]:
#find and list all possible years

years = explicit["year"].unique().tolist()
years.sort()
years

[1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [41]:
#Create a list, with years as 'key', and genre as key within the key with counts of genres as values for exT
#1. create new list 'mylistT', 2. assign var. 'max' to count number of years found (so it knows how often
#- it must iterate) 3.Iterate through every year 4. use before found loop to link values found to years. 

mylistT = []
max = len(years)
for year in range(max):
    #print (year,years[year])
    exT = explicit[(explicit["year"] == years[year])]
    exT = exT[(exT["explicit"] == True)]
    exT = exT.reset_index()
    
    frequency = {}
    max2 = len(exT)
    for x in range(max2):
        items = re.split(",", exT.loc[int(x):int(x),"genre"].values[0])
        #print (x,items)
        for item in items:
            #print (item)
            if item in frequency:
                frequency[item] += 1
            else:
                frequency[item] = 1
    
    mylistT.append([years[year],frequency])

    print(mylistT[year])
    
    

[1998, {'hip hop': 1, ' pop': 1, ' R&B': 1}]
[1999, {'hip hop': 3, ' pop': 2, ' R&B': 1, 'set()': 1}]
[2000, {'hip hop': 4, ' pop': 3, ' R&B': 1}]
[2001, {'hip hop': 3, ' pop': 1}]
[2002, {'hip hop': 3, ' pop': 3, ' R&B': 1}]
[2003, {'hip hop': 3, ' pop': 3, ' R&B': 1}]
[2004, {'pop': 1, 'hip hop': 3, ' pop': 2, ' R&B': 1, 'rock': 1}]
[2005, {'hip hop': 4, ' pop': 3, 'pop': 1}]
[2006, {'hip hop': 1, ' pop': 1, 'pop': 2, ' R&B': 1}]
[2007, {'hip hop': 2, ' pop': 2, ' R&B': 1}]
[2008, {'hip hop': 2, ' pop': 2, 'R&B': 1, ' R&B': 1}]
[2009, {'hip hop': 2, ' pop': 3, ' R&B': 1, 'Folk/Acoustic': 1, ' rock': 1, 'Dance/Electronic': 1}]
[2010, {'hip hop': 6, ' pop': 6, ' Dance/Electronic': 1}]
[2011, {'hip hop': 4, ' pop': 3, 'Dance/Electronic': 2, ' Dance/Electronic': 1}]
[2012, {'pop': 2, 'hip hop': 5, ' Dance/Electronic': 1, ' pop': 4, ' R&B': 1}]
[2013, {'pop': 1, 'hip hop': 4, ' pop': 1, ' R&B': 1}]
[2014, {'hip hop': 5, ' pop': 5, ' R&B': 1, ' Dance/Electronic': 1, 'pop': 1}]
[2015, {'pop

In [42]:
mylistT


[[1998, {'hip hop': 1, ' pop': 1, ' R&B': 1}],
 [1999, {'hip hop': 3, ' pop': 2, ' R&B': 1, 'set()': 1}],
 [2000, {'hip hop': 4, ' pop': 3, ' R&B': 1}],
 [2001, {'hip hop': 3, ' pop': 1}],
 [2002, {'hip hop': 3, ' pop': 3, ' R&B': 1}],
 [2003, {'hip hop': 3, ' pop': 3, ' R&B': 1}],
 [2004, {'pop': 1, 'hip hop': 3, ' pop': 2, ' R&B': 1, 'rock': 1}],
 [2005, {'hip hop': 4, ' pop': 3, 'pop': 1}],
 [2006, {'hip hop': 1, ' pop': 1, 'pop': 2, ' R&B': 1}],
 [2007, {'hip hop': 2, ' pop': 2, ' R&B': 1}],
 [2008, {'hip hop': 2, ' pop': 2, 'R&B': 1, ' R&B': 1}],
 [2009,
  {'hip hop': 2,
   ' pop': 3,
   ' R&B': 1,
   'Folk/Acoustic': 1,
   ' rock': 1,
   'Dance/Electronic': 1}],
 [2010, {'hip hop': 6, ' pop': 6, ' Dance/Electronic': 1}],
 [2011,
  {'hip hop': 4, ' pop': 3, 'Dance/Electronic': 2, ' Dance/Electronic': 1}],
 [2012,
  {'pop': 2, 'hip hop': 5, ' Dance/Electronic': 1, ' pop': 4, ' R&B': 1}],
 [2013, {'pop': 1, 'hip hop': 4, ' pop': 1, ' R&B': 1}],
 [2014,
  {'hip hop': 5, ' pop': 5, ' 

In [62]:
Falsefreq = {}
loop = len(exF)
for y in range(loop):
    values = re.split(",", exF.loc[int(y):int(y),"genre"].values[0])
    #print(y, values)
    for value in values:
        #print(value)
        if value in Falsefreq:
            Falsefreq[value] += 1
        else:
            Falsefreq[value] = 1

In [68]:
#Falsefreq

In [70]:
mylistF =[]
max = len(years)
for year in range(max):
    #print(year, years[year])
    exF = explicit[(explicit["year"] == years[year])]
    exF = exF[(exF["explicit"] == True)]
    exF = exF.reset_index()
    #print(exF)
    
    Falsefreq = {}
    loop = len(exF)
    for y in range(loop):
        values = re.split(",", exF.loc[int(y):int(y),"genre"].values[0])
        #print(y, values)
        for value in values:
            #print(value)
            if value in Falsefreq:
                Falsefreq[value] += 1
            else:
                Falsefreq[value] = 1
    
    mylistF.append([years[year],Falsefreq])
    
    print(mylistF[year])

[1998, {'hip hop': 1, ' pop': 1, ' R&B': 1}]
[1999, {'hip hop': 3, ' pop': 2, ' R&B': 1, 'set()': 1}]
[2000, {'hip hop': 4, ' pop': 3, ' R&B': 1}]
[2001, {'hip hop': 3, ' pop': 1}]
[2002, {'hip hop': 3, ' pop': 3, ' R&B': 1}]
[2003, {'hip hop': 3, ' pop': 3, ' R&B': 1}]
[2004, {'pop': 1, 'hip hop': 3, ' pop': 2, ' R&B': 1, 'rock': 1}]
[2005, {'hip hop': 4, ' pop': 3, 'pop': 1}]
[2006, {'hip hop': 1, ' pop': 1, 'pop': 2, ' R&B': 1}]
[2007, {'hip hop': 2, ' pop': 2, ' R&B': 1}]
[2008, {'hip hop': 2, ' pop': 2, 'R&B': 1, ' R&B': 1}]
[2009, {'hip hop': 2, ' pop': 3, ' R&B': 1, 'Folk/Acoustic': 1, ' rock': 1, 'Dance/Electronic': 1}]
[2010, {'hip hop': 6, ' pop': 6, ' Dance/Electronic': 1}]
[2011, {'hip hop': 4, ' pop': 3, 'Dance/Electronic': 2, ' Dance/Electronic': 1}]
[2012, {'pop': 2, 'hip hop': 5, ' Dance/Electronic': 1, ' pop': 4, ' R&B': 1}]
[2013, {'pop': 1, 'hip hop': 4, ' pop': 1, ' R&B': 1}]
[2014, {'hip hop': 5, ' pop': 5, ' R&B': 1, ' Dance/Electronic': 1, 'pop': 1}]
[2015, {'pop